In [1]:
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
train = pd.read_csv('./train.csv')

In [3]:
train.head()

label                                               text
0      2  Gas by my house hit $3.39!!!! I'm going to Cha...
1      0  Iranian general says Israel's Iron Dome can't ...
2      2  with J Davlar 11th. Main rivals are team Polan...
3      0  Talking about ACT's &amp;&amp; SAT's, deciding...
4      0  They may have a SuperBowl in Dallas, but Dalla...

In [4]:
x_train = train['text']
y_train = np.array(train['label'])

In [ ]:
import re
from nltk.corpus import stopwords

stopwords_eng = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower() # convert to lowercase
    text = re.sub("[^a-z]", " ", text)
    # words = [word for word in text.split() if word not in stopwords_eng]
    words = [word for word in text.split()]
    text = " ".join(words)
    return text

x_train = x_train.map(clean_text)
x_train.head()

0    gas by my house hit i m going to chapel hill o...
1    iranian general says israel s iron dome can t ...
2    with j davlar th main rivals are team poland h...
3    talking about act s amp amp sat s deciding whe...
4    they may have a superbowl in dallas but dallas...
Name: text, dtype: object

In [ ]:
from keras.preprocessing.text import Tokenizer
num_words = 6000
tokenizer = Tokenizer(num_words=num_words)

tokenizer.fit_on_texts(x_train)

x_train_seq = pd.Series(tokenizer.texts_to_sequences(x_train))
x_train_seq.head()

0    [2422, 70, 23, 203, 402, 3, 35, 33, 2, 2423, 1...
1    [1975, 283, 1078, 13, 2424, 3997, 42, 4, 660, ...
2    [19, 395, 22, 929, 4752, 41, 181, 1976, 539, 3...
3    [928, 65, 640, 13, 52, 52, 119, 13, 3468, 201,...
4    [75, 28, 25, 7, 1209, 5, 523, 38, 523, 1134, 4...
dtype: object

In [ ]:
x_train_len = x_train_seq.map(lambda ls: len(ls))
x_train_len.describe()

count    10026.000000
mean        18.052663
std          5.524935
min          1.000000
25%         14.000000
50%         19.000000
75%         22.000000
max         67.000000
dtype: float64

In [ ]:
from keras.preprocessing.sequence import pad_sequences

x_train_pad = pad_sequences(x_train_seq, maxlen=512)

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GlobalMaxPool1D, Dropout, Activation

model = 0
model = Sequential()

model.add(Embedding(input_dim=num_words, output_dim=64))
model.add(LSTM(8, return_sequences=True))
model.add(GlobalMaxPool1D())
#model.add(Dense(16, activation='tanh'))
model.add(Dropout(0.8))
model.add(Dense(3, activation='softmax'))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          384000    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 8)           2336      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 8)                 0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 27        
Total params: 386,363
Trainable params: 386,363
Non-trainable params: 0
_______________

In [ ]:
y_train = keras.utils.to_categorical(y_train,num_classes=3)

In [ ]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

batch_size = 32
epochs = 10
validation_split = 0.2
model.fit(x=x_train_pad, y=y_train, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8020 samples, validate on 2006 samples
Epoch 1/10
6880/8020 [========================>.....] - ETA: 47s - loss: 0.6101 - acc: 0.6693

In [ ]:
test = pd.read_csv("./sentence-sentiment-analysis/test.tsv", delimiter="\t", quoting=3, header=0)
test.head()

In [ ]:
test = test.text

In [ ]:
x_test = test.map(clean_text)
x_test.head()

In [ ]:
x_test_seq = tokenizer.texts_to_sequences(x_test)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)

In [ ]:
pred = model.predict(x=x_test_pad)

In [ ]:
v = np.argmax(pred, axis=1)

In [ ]:
v

In [ ]:
p = []
for i in v:
    if i == 0:
        p.append('negative')
    elif i == 1:
        p.append('neutral')
    else:
        p.append('positive')

In [ ]:
df = pd.DataFrame({'polarity':p})
df.index += 1
df.to_csv('sub.csv', index = True, index_label = 'id')